# Which csv?

## database.csv or fatalencounters??? or both??

We are going to try to see if we can use the name as a temporary key to mix the two databases

In [14]:
# Importing pandas

import pandas as pd
import datetime as dt

In [7]:
# Importing the 2 databases

wpost = pd.read_csv ('/Users/ReDi/Documents/GitHub/data-analysis-project/data/database.csv')
fenc = pd.read_csv ('/Users/ReDi/Documents/GitHub/data-analysis-project/data/fatalencounters.csv')

In [8]:
wpost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 14 columns):
id                         2142 non-null int64
name                       2123 non-null object
date                       2142 non-null object
manner_of_death            2142 non-null object
armed                      2136 non-null object
age                        2099 non-null float64
gender                     2141 non-null object
race                       2039 non-null object
city                       2142 non-null object
state                      2142 non-null object
signs_of_mental_illness    2142 non-null bool
threat_level               2142 non-null object
flee                       2106 non-null object
body_camera                2142 non-null bool
dtypes: bool(2), float64(1), int64(1), object(10)
memory usage: 205.1+ KB


In [9]:
fenc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20792 entries, 0 to 20791
Data columns (total 24 columns):
Timestamp                                                                          20791 non-null float64
Subject's name                                                                     20792 non-null object
Subject's age                                                                      20207 non-null object
Subject's gender                                                                   20757 non-null object
Subject's race                                                                     20791 non-null object
URL of image of deceased                                                           7084 non-null object
Date of injury resulting in death (month/day/year)                                 20792 non-null object
Location of injury (address)                                                       20315 non-null object
Location of death (city)                           

In [10]:
wpost.describe()

,id,age
count,2142.00000,2099.000000
mean,1232.21662,36.484516
std,670.62817,12.874000
min,3.00000,6.000000
25%,664.50000,26.000000
50%,1231.50000,34.000000
75%,1811.75000,45.000000
max,2394.00000,86.000000


In [12]:
wpost.describe(include = ['O'])

,name,date,manner_of_death,armed,gender,race,city,state,threat_level,flee
count,2123,2142,2142,2136,2141,2039,2142,2142,2142,2106
unique,2118,739,2,64,2,6,1247,51,3,4
top,Daquan Antonio Westbrook,2015-12-14,shot,gun,M,W,Los Angeles,CA,attack,Not fleeing
freq,2,8,1992,1173,2052,1041,31,355,1381,1453


In [13]:
fenc.describe()

,Timestamp,20791,Unique identifier,Date (Year)
count,20791.000000,32.000000,20791.000000,20792.000000
mean,10396.000000,19435.468750,10396.000000,2009.507984
std,6001.989059,248.357181,6001.989059,4.976123
min,1.000000,19279.000000,1.000000,2000.000000
25%,5198.500000,19387.750000,5198.500000,2005.000000
50%,10396.000000,19395.500000,10396.000000,2010.000000
75%,15593.500000,19403.250000,15593.500000,2014.000000
max,20791.000000,20791.000000,20791.000000,2100.000000


In [15]:
fenc.describe ( include = ['O'])

,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),Location of death (zip code),Location of death (county),Agency responsible for death,Cause of death,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description
count,20792,20207,20757,20791,7084,20792,20315,20755,20791,20612,20778,20745,20791,20791,20765,20790,20732,15081,20792,20757
unique,20041,122,3,8,7008,5974,19121,4891,51,8590,1363,5829,15,19830,54,19293,4,827,3,19868
top,Name withheld by police,22,Male,Race unspecified,http://pursuitsafety.org/images/4_killed_phill...,"February 19, 2003",2347 S. Michigan Ave.,Chicago,CA,60616,Los Angeles,Chicago Police Department,Gunshot,"On February 19, 2003, 21 people were killed an...",Unreported,http://www.usatoday.com/pages/interactives/bla...,No,CRDeCeros,Complete,"2/19/2003: On February 19, 2003, 21 people wer..."
freq,593,730,18827,8061,3,23,21,401,3868,31,1045,373,14809,21,9091,132,12795,3604,19505,21


In [21]:
dt.datetime.strptime('February 09, 2003','%B %d, %Y')

datetime.datetime(2003, 2, 9, 0, 0)

In [23]:
for 
fenc['date'] = dt.datetime.strftime(fenc['Date of injury resulting in death (month/day/year)'],'%B %d, %Y')

TypeError: descriptor 'strftime' requires a 'datetime.date' object but received a 'Series'